# Multicall and Regular RPC mix 

MakerDAO's Multicall Smart Contrats https://github.com/makerdao/multicall \
JS original https://github.com/makerdao/multicall.js \
Python port https://github.com/banteg/multicall.py \
\
Multical smartcontracts are not available on some blockchains, so below is demo of the combination of Multicall and Regular RPC methods. \
Multicall contracs:
https://github.com/banteg/multicall.py/blob/master/multicall/constants.py

In [ ]:
pip install multicall

### Example Input Data Frame (Infura API Key required)

In [80]:
import pandas as pd

Infura_API_Key = 'Infura_API_Key'#Ethereum only needed
assert Infura_API_Key != 'Infura_API_Key', "Infura API Key required!"

addr = {'blockchain': ['ethereum', 'ethereum', 'ethereum','matic','matic','unknown']
    ,'address': ['0x188407eeD7B1bb203dEd6801875C0B5Cb1027053'
                 ,'0x1a66f065303299d78693f122c800Ab3dEbE9c966'
                 ,'0x1e0447b19bb6ecfdae1e4ae1694b0c3659614e4e'
                 ,'0x1a13F4Ca1d028320A707D99520AbFefca3998b7F'
                 ,'0x1a13F4Ca1d028320A707D99520AbFefca3998b7F'
                ,'WrongAddress']
       }
addr_df =  pd.DataFrame(data = addr) 

tick = {'blockchain': ['ethereum', 'ethereum', 'ethereum','matic','matic','unknown']
    ,'contract_ticker_symbol': ['FRONT', 'DAI', 'SAI','USDC','USDT','DUMMY']
    ,'contract_name': ['TokenName1', 'TokenName2', 'TokenName3','TokenName4','TokenName5','TokenName6']
    ,'contract_address': ['0xf8c3527cc04340b208c854e985240c02f7b7793f'
                          ,'0x6b175474e89094c44da98b954eedeac495271d0f'
                          ,'0x89d24a6b4ccb1b6faa2625fe562bdd9a23260359'
                          ,'0x2791bca1f2de4661ed88a30c99a7a9449aa84174'
                          ,'0xc2132d05d31c914a87c6611c10748aeb04b58e8f'
                          ,'xxWrongAddress']
    }
tick_df =  pd.DataFrame(data = tick) 

in_df = addr_df.merge(tick_df, how='outer')

in_df['apikey'] = Infura_API_Key 
display(in_df)


,blockchain,address,contract_ticker_symbol,contract_name,contract_address,apikey
0,ethereum,0x188407eeD7B1bb203dEd6801875C0B5Cb1027053,FRONT,TokenName1,0xf8c3527cc04340b208c854e985240c02f7b7793f,Infura_API_Key
1,ethereum,0x188407eeD7B1bb203dEd6801875C0B5Cb1027053,DAI,TokenName2,0x6b175474e89094c44da98b954eedeac495271d0f,Infura_API_Key
2,ethereum,0x188407eeD7B1bb203dEd6801875C0B5Cb1027053,SAI,TokenName3,0x89d24a6b4ccb1b6faa2625fe562bdd9a23260359,Infura_API_Key
3,ethereum,0x1a66f065303299d78693f122c800Ab3dEbE9c966,FRONT,TokenName1,0xf8c3527cc04340b208c854e985240c02f7b7793f,Infura_API_Key
4,ethereum,0x1a66f065303299d78693f122c800Ab3dEbE9c966,DAI,TokenName2,0x6b175474e89094c44da98b954eedeac495271d0f,Infura_API_Key
5,ethereum,0x1a66f065303299d78693f122c800Ab3dEbE9c966,SAI,TokenName3,0x89d24a6b4ccb1b6faa2625fe562bdd9a23260359,Infura_API_Key
6,ethereum,0x1e0447b19bb6ecfdae1e4ae1694b0c3659614e4e,FRONT,TokenName1,0xf8c3527cc04340b208c854e985240c02f7b7793f,Infura_API_Key
7,ethereum,0x1e0447b19bb6ecfdae1e4ae1694b0c3659614e4e,DAI,TokenName2,0x6b175474e89094c44da98b954eedeac495271d0f,Infura_API_Key
8,ethereum,0x1e0447b19bb6ecfdae1e4ae1694b0c3659614e4e,SAI,TokenName3,0x89d24a6b4ccb1b6faa2625fe562bdd9a23260359,Infura_API_Key
9,matic,0x1a13F4Ca1d028320A707D99520AbFefca3998b7F,USDC,TokenName4,0x2791bca1f2de4661ed88a30c99a7a9449aa84174,Infura_API_Key


### Main Code

In [39]:
from web3 import Web3
import sys
import pandas as pd
import numpy as np
import time
import requests
from decimal import Decimal
from web3.datastructures import AttributeDict
from multicall import Call, Multicall
from multicall.constants import MULTICALL_ADDRESSES

abi = """
[ { "constant":true, "inputs":[], "name":"name", "outputs":[ { "name":"", "type":"string" } ], "payable":false, "type":"function" }, { "constant":true, "inputs":[], "name":"decimals", "outputs":[ { "name":"", "type":"uint8" } ], "payable":false, "type":"function" }, { "constant":true, "inputs":[ { "name":"_owner", "type":"address" } ], "name":"balanceOf", "outputs":[ { "name":"balance", "type":"uint256" } ], "payable":false, "type":"function" }, { "constant":true, "inputs":[], "name":"symbol", "outputs":[ { "name":"", "type":"string" } ], "payable":false, "type":"function" } ]
"""
ChainID = {'ethereum': 1
               ,'matic':137
               ,'polygon':137
               ,'bsc':56
               ,'heco':128
               ,'avalanche':43114
               ,'fantom':250
               ,'xdai':100
}

Endpoint = { 'ethereum': f'https://mainnet.infura.io/v3/{in_df.loc[0,"apikey"]}' 
            ,'matic':'https://polygon-rpc.com'
            ,'polygon':'https://polygon-rpc.com'
            ,'bsc':'https://bsc-dataseed.binance.org/'
            ,'heco':'https://http-mainnet.hecochain.com'
            ,'avalanche':'https://api.avax.network/ext/bc/C/rpc'
            ,'fantom':'https://rpcapi.fantom.network'
            ,'xdai': 'https://rpc.xdaichain.com'
           }
NativeToken = {'ethereum': 'ETH'
               ,'matic':'MATIC'
               ,'polygon':'MATIC'
               ,'bsc':'BNB'
               ,'heco':'HT'
               ,'avalanche':'AVAX'
               ,'fantom':'FTM'
               ,'xdai':'XDAI'
              }

def get_w3(row):
    native_token = ''
    w3 = None
    if row['blockchain'] in Endpoint:
        w3 = Web3(Web3.HTTPProvider(Endpoint[row['blockchain']], request_kwargs={'timeout': 60}))
    if row['blockchain'] in NativeToken:
        native_token = NativeToken[row['blockchain']] 
        
    if row['blockchain'] == 'ethereum':
        if row['address'] == row['contract_address']:
            row['contract_address'] = '0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee'
    elif row['blockchain'] == 'bsc':
        if row['address'] == row['contract_address']:
            row['contract_address'] = '0xb8c77482e45f1f44de1745f52c74426c631bdd52'
              
    if (row['address'] == row['contract_address']) or (row['contract_address'] =='0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee') or (row['blockchain'] =='bsc' and row['contract_address'] =='0xb8c77482e45f1f44de1745f52c74426c631bdd52'):
        row['contract_ticker_symbol'] = native_token      
    return w3, row, native_token

def get_token_balance(df):
    for i, row in df.iterrows():

        w3, row, native_token = get_w3(row)
        df.loc[i, :] = row
        if w3:
            balance = 0
            decimals = 18

            try:
                address = Web3.toChecksumAddress(row['address'])
                contract_address = Web3.toChecksumAddress(row['contract_address'])

                if row['contract_ticker_symbol'] == native_token:
                    balance = Decimal(w3.eth.get_balance(address))
                else:
                    contract = w3.eth.contract(address=contract_address, abi=abi)
                    balance = Decimal(contract.functions.balanceOf(address).call())
                    decimals = contract.functions.decimals().call()
            except:
                df.loc[i, 'error'] = sys.exc_info()[0]

            df.loc[i, ['balance', 'decimals']] = [balance, decimals]
    return df


def get_multibalance(df, blockchain):
    icnt = 0
    if len(df) == 0:
        return 
    w3 = Web3(Web3.HTTPProvider(Endpoint[blockchain], request_kwargs={'timeout': 60}))
    BalanceOfString = 'balanceOf(address)(uint256)'
    DecimalsString = 'decimals()(uint8)'
    multi_balance = Multicall([
                    Call(y, [BalanceOfString, x], [[x+':'+y, Decimal]]) for x,y in zip(df['address'], df['contract_address'])
                    ]
                    ,_w3 = w3)
    try:
        multi_balance = multi_balance()
    except (requests.exceptions.HTTPError):
        time.sleep(3)
        multi_balance = multi_balance()
        
    d_multi_balance = AttributeDict.recursive(multi_balance)
    
    multi_decimals = Multicall([
                    Call(y, DecimalsString, [[x+':'+y, None]]) for x,y in zip(df['address'], df['contract_address'])
                    ]
                    ,_w3 = w3)
    multi_decimals = multi_decimals()
    d_multi_decimals = AttributeDict.recursive(multi_decimals)
    
    df.loc[:,'balance'] = df.apply(lambda x: d_multi_balance[x['address']+':'+x['contract_address']], axis=1)
    df.loc[:,'decimals'] = df.apply(lambda x: d_multi_decimals[x['address']+':'+x['contract_address']], axis=1)
    
    return df 


out_df_init = in_df.copy()
out_df_init = out_df_init[out_df_init['address'].apply(Web3.isAddress)]

cols = list(out_df_init.columns.values)
cols.remove('contract_address')
cols.remove('contract_ticker_symbol')
cols.remove('contract_name')
out_df_native = out_df_init.groupby('address')[cols].agg(np.unique)
out_df_native['contract_ticker_symbol'] ='-'
out_df_native['contract_address'] = out_df_native['address'] 
out_df_native.reset_index(drop=True, inplace=True)

out_df_native = get_token_balance(out_df_native)

out_df = out_df_native
for blockchain in Endpoint.keys():
    df = out_df_init.loc[out_df_init['blockchain'] == blockchain].copy()
    if ChainID[blockchain] in MULTICALL_ADDRESSES:
        try:
            df = get_multibalance(df, blockchain)
        except:
            df = get_token_balance(df)
    else:
        df = get_token_balance(df)
    
    out_df = pd.concat([out_df, df], ignore_index = True)

if len(out_df) == 0:
    out_df.loc[len(out_df), ['error']] = ['empty']
else:
    out_df['balance'] = out_df['balance'].astype('str')	   


### Output: balance (native and token), decimals

In [67]:
reverse_cols = list(out_df.columns)
reverse_cols.reverse()
reverse_cols.remove('apikey')
display(out_df[reverse_cols])

,contract_name,decimals,balance,contract_address,contract_ticker_symbol,address,blockchain
0,NaN,18.0,641709818647583598,0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee,ETH,0x188407eeD7B1bb203dEd6801875C0B5Cb1027053,ethereum
1,NaN,18.0,0,0x1a13F4Ca1d028320A707D99520AbFefca3998b7F,MATIC,0x1a13F4Ca1d028320A707D99520AbFefca3998b7F,matic
2,NaN,18.0,0,0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee,ETH,0x1a66f065303299d78693f122c800Ab3dEbE9c966,ethereum
3,NaN,18.0,0,0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee,ETH,0x1e0447b19bb6ecfdae1e4ae1694b0c3659614e4e,ethereum
4,TokenName1,18.0,5143488518400000000000,0xf8c3527cc04340b208c854e985240c02f7b7793f,FRONT,0x188407eeD7B1bb203dEd6801875C0B5Cb1027053,ethereum
5,TokenName2,18.0,0,0x6b175474e89094c44da98b954eedeac495271d0f,DAI,0x188407eeD7B1bb203dEd6801875C0B5Cb1027053,ethereum
6,TokenName3,18.0,0,0x89d24a6b4ccb1b6faa2625fe562bdd9a23260359,SAI,0x188407eeD7B1bb203dEd6801875C0B5Cb1027053,ethereum
7,TokenName1,18.0,0,0xf8c3527cc04340b208c854e985240c02f7b7793f,FRONT,0x1a66f065303299d78693f122c800Ab3dEbE9c966,ethereum
8,TokenName2,18.0,0,0x6b175474e89094c44da98b954eedeac495271d0f,DAI,0x1a66f065303299d78693f122c800Ab3dEbE9c966,ethereum
9,TokenName3,18.0,0,0x89d24a6b4ccb1b6faa2625fe562bdd9a23260359,SAI,0x1a66f065303299d78693f122c800Ab3dEbE9c966,ethereum
